# Provenance & License

**Source file:** `https://github.com/Yvictor/TradingGym/blob/master/README.md`   
**Original author:** Yvictor
**License:** MIT License (see upstream repository)

**Adapted by:**

**Changes:**
- 

**Link to source:**  
https://github.com/Yvictor/TradingGym/blob/master/README.md


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import trading_env
import os

In [2]:
import numpy as np
import pandas as pd

def ohlcv_to_tradinggym(df_ohlcv: pd.DataFrame) -> pd.DataFrame:
    df = df_ohlcv.copy()

    df["datetime"] = pd.to_datetime(df["date"], utc=False)
    df = df.sort_values("datetime").dropna(subset=["close", "volume"])

    df["Price"]  = df["close"].astype(float)
    df["Volume"] = df["volume"].astype(float)

    spread = df["Price"] * 0.0001
    df["Ask_price"] = (df["Price"] + spread/2).astype(float)
    df["Bid_price"] = (df["Price"] - spread/2).astype(float)

    updown = np.sign(df["Price"].diff()).fillna(0.0)
    df["Updown"] = updown.astype(float)
    df["Updown_Cum"] = df["Updown"].cumsum()

    ask_ratio = np.where(df["Updown"] > 0, 0.7, np.where(df["Updown"] < 0, 0.3, 0.5))
    df["Ask_deal_vol"] = (df["Volume"] * ask_ratio).astype(float)
    df["Bid_deal_vol"] = (df["Volume"] - df["Ask_deal_vol"]).astype(float)

    df["Bid/Ask_deal"] = (df["Bid_deal_vol"] - df["Ask_deal_vol"]) / (
        (df["Bid_deal_vol"] + df["Ask_deal_vol"]).replace(0, np.nan)
    )
    df["Bid/Ask_deal"] = df["Bid/Ask_deal"].fillna(0.0)

    df["serial_number"] = np.arange(len(df), dtype=int)
    df["Comm"] = "CUSTOM"

    cols = [
        "serial_number","datetime","Comm","Price","Volume",
        "Ask_price","Bid_price","Ask_deal_vol","Bid_deal_vol",
        "Bid/Ask_deal","Updown","Updown_Cum"
    ]
    return df[cols]


In [3]:
train = pd.read_csv('../data/train_data.csv')
df = ohlcv_to_tradinggym(train)
df

FileNotFoundError: [Errno 2] No such file or directory: '../data/train_data.csv'

In [ ]:

env = trading_env.make(env_id='backtest_v1', obs_data_len=10, step_len=1,
                       df=df, fee=0.1, max_position=5, deal_col_name='Price', 
                       feature_names=['Price'])

class RandomAgent:
    def __init__(self):
        pass
        
    def choice_action(self, state):
        return np.random.randint(3)


agent = RandomAgent()

transactions = []
while not env.backtest_done:
    state = env.reset()
    done = False
    while not done:
        state, reward, done, info = env.step(agent.choice_action(state))
        print(reward)
        #env.render()
        if done:
            transactions.append(info)
            break
transaction = pd.concat(transactions)
transaction.to_csv(os.path.join("./results", "Trading_gym.csv"))


realized = env.info["reward_fluctuant"] * (env.info["reward_makereal"] != 0).astype(float)
initial_cash   = 100_000.0
contract_size  = 1.0

equity_curve = initial_cash + (realized.fillna(0.0) * contract_size).cumsum()
start_money  = equity_curve.iloc[0]
end_money    = equity_curve.iloc[-1]

print(f"Start: {start_money:.2f}")
print(f"End:   {end_money:.2f}")
print(f"Return: {(end_money/start_money - 1)*100:.2f}%")

print("Full transaction saved to Trading_gym.csv")





[2025-10-24 17:35:42,455] Making new env: backtest_v1


0.0
0.0
-21.33469734191895
-21.33469734191895
-38.62860794067383
-38.62860794067383
-20.49997310638428
-20.49997310638428
-3.422558689117441
-3.422558689117441
-18.56159138679505
-18.56159138679505
25.892199397087097
28.807849311828615
46.1017599105835
-3.0464817047119084
-21.175116539001458
-21.175116539001458
-38.2525309562683
-38.2525309562683
-23.113498258590692
-23.113498258590692
-46.23259170055389
-60.99713034629822
-60.99713034629822
-28.026049327850345
-26.214306879043587
-26.214306879043587
-38.32629778385163
-38.32629778385163
-38.32629778385163
-38.32629778385163
-28.21865344047547
-32.08706486225128
-32.08706486225128
-1.4232125282287598
-3.2349549770355193
-3.2349549770355193
8.877035927772525
8.877035927772525
8.877035927772525
8.877035927772525
-23.38885517120361
-7.671555018424987
-5.776725192070007
-20.263416929244997
-23.73095257377625
-23.73095257377625
-23.730952573776246
-22.64756409721375
-17.995168700790416
-17.99516870079041
4.163078054809566
-8.865172433090216